# All training data

In [1]:
%config Completer.use_jedi = False

### 
- now using 
    - 20211029_011638: ./dir_kicker/logs/files__all_train_kfold_20211029_031110.csv


- all data
    - (Don't use) 20211025_012816: ./logs/files_20211025_020358_alldata.csv
    - (Use) files_20220212_200237_alldata.csv
    
        - or 
            - scripts/dir_kicker/logs/kicker.sh.o205129
            - 

In [2]:
!ls ../scripts/dir_kicker/logs/

files_20211025_001501_mcpas.csv
files_20211025_012422_alltrain.csv
files_20211025_020358_alldata.csv
files_20220212_200237_alldata.csv
files__all_train_kfold_20211029_031110.csv
files_fake020211027_051219.csv
kicker.sh.o205127
kicker.sh.o205128
kicker.sh.o205129
kicker.sh.o205130
kicker.sh.o205131
kicker.sh.o365034
kicker.sh.o380085


In [3]:
from analysis_util import pdread
import pandas as pd
from scipy.special import expit, softmax
from sklearn.metrics import roc_auc_score
import os, sys
import joblib, functools
from functools import partial
import pickle
from sklearn.model_selection import KFold

sys.path.append('../recipes/')
import dataset
MCPASDataset = dataset.MCPASDataset


def get_file_paths(bucket, dirname, feat_name, extension='.parquet'):
    import boto3
    dataroot = f"s3://{bucket}/"
    objs = boto3.resource("s3").Bucket(f"{bucket}").objects.filter(
        Prefix=f"{dirname}/{feat_name}")
    files = [obj.key for obj in objs
             if not obj.key.startswith(f"{dirname}.ipynb_checkpoints") and obj.key.endswith(extension)]
    files = [os.path.join(dataroot, f) for f in files]
    return files


def delete_padding(weights1):
    if len([c for c in weights1.columns if '8_' in c])>0: 
        weights1 = weights1.drop(columns=[c for c in weights1.columns if '8_' in c])
    if len([c for c in weights1.index if '8_' in c])>0: 
        weights1 = weights1.drop(index=[c for c in weights1.index if '8_' in c])
    return weights1

@functools.lru_cache(maxsize=10)
def get_data_i_from_attenmatdict(ind):
    '''Get the data: attention, tcra, tcrb, peptides, sign etc'''
    attn_output_weights1, attn_output_weights2, abseq_with_comma, peptide, ypred, sign = ATTENTION_MATRIX_DICT[ind]
    aseq_from_comma, bseq_from_comma =  abseq_with_comma.split(':')
    aseq = DF_TCR.iloc[ind]['tcra']
    bseq = DF_TCR.iloc[ind]['tcrb']
    assert aseq==aseq_from_comma
    assert bseq==bseq_from_comma
    abseq = convert_len(aseq, MAXLENGTH_A) + convert_len(bseq, MAXLENGTH_B) 
    peptide = convert_len(DF_TCR.iloc[ind]['peptide'], max_len_epitope)
    return attn_output_weights1, attn_output_weights2, abseq_with_comma, peptide, ypred, sign, aseq, bseq, abseq, peptide


def get_weight1_on_headi(attn_output_weights1, head_i, 
                                   aseq, maxlength_a, 
                                   bseq, maxlength_b, 
                                   peptide,
                                   sign
                                  ):
    weights1 = pd.DataFrame(attn_output_weights1[head_i])
    weights1 = pd.DataFrame(attn_output_weights1[head_i])
    weights1.index = [f"{tup[1]}_{tup[0]}" for tup in list(enumerate(convert_len(aseq, maxlength_a)))] +\
        [f"{tup[1]}_{tup[0]}" for tup in list(enumerate(convert_len(bseq, maxlength_b)))]
    weights1.columns = [f"{tup[1]}_{tup[0]}" for tup in list(enumerate(peptide))]

    weights1 = delete_padding(weights1)
    weights1 = weights1.melt(ignore_index=False).reset_index().rename(columns={'variable':'peptide', 'index':'tcr'})
    weights1['sign'] = sign
    weights1['atten_head'] = head_i
    return weights1


def get_weight2_on_headi(attn_output_weights2, head_i, 
                           aseq, maxlength_a, 
                           bseq, maxlength_b,
                           peptide,
                           sign):
    weights2 = pd.DataFrame(attn_output_weights2[head_i])
    weights2.index = [f"{tup[1]}_{tup[0]}" for tup in list(enumerate(peptide))]
    weights2.columns = [f"{tup[1]}_{tup[0]}" for tup in list(enumerate(convert_len(aseq, maxlength_a)))] + \
        [f"{tup[1]}_{tup[0]}" for tup in list(enumerate(convert_len(bseq, maxlength_b)))]
    weights2 = delete_padding(weights2)
    weights2 = weights2.melt(ignore_index=False).reset_index().rename(columns={'variable':'tcr', 'index':'peptide'})
    weights2['sign'] = sign
    weights2['atten_head'] = head_i
    return weights2



def get_attention_weights_from_index(attention_matrix_dict, index):
    weights1_list = []
    weights2_list = []

    for head_i in range(4):
        for ind in tqdm(index):            
            attn_output_weights1, attn_output_weights2, abseq_with_comma, peptide, ypred, sign, aseq, bseq, abseq, peptide = \
                get_data_i_from_attenmatdict(ind)

            weights1 = get_weight1_on_headi(attn_output_weights1, head_i, 
                                   aseq, MAXLENGTH_A, 
                                   bseq, MAXLENGTH_B,
                                   peptide,
                                   sign
                                  )
            
            weights1_list.append(weights1)
            
            weights2 = get_weight2_on_headi(attn_output_weights2, head_i, 
                                   aseq, MAXLENGTH_A, 
                                   bseq, MAXLENGTH_B,
                                   peptide,
                                   sign)
            
            weights2_list.append(weights2)

        weights1_total = pd.concat(weights1_list)
        weights1_total['type'] = f'weights1_total: sum1onPeptide'
        weights2_total = pd.concat(weights2_list)
        weights2_total['type'] = f'weights2_total: sum1onTCR'
    df_attention_weights = pd.concat([weights1_total, weights2_total])
    _split_tcr = pd.DataFrame(df_attention_weights['tcr'].str.split('_', 1).tolist())
    _split_tcr.columns = ['tcr', 'position_of_tcr']
    _split_tcr['position_of_tcr'] = _split_tcr['position_of_tcr'].astype(int)
    _split_peptide = pd.DataFrame(df_attention_weights['peptide'].str.split('_', 1).tolist())
    _split_peptide.columns = ['peptide', 'position_of_peptide']
    _split_peptide['position_of_peptide'] = _split_peptide['position_of_peptide'].astype(int)
    
    df_attention_weights = df_attention_weights.rename(columns={'tcr':'tcr_position', 'peptide':'peptide_position'})
    df_attention_weights = pd.concat([df_attention_weights.reset_index(drop=True), 
               _split_tcr.reset_index(drop=True), 
               _split_peptide.reset_index(drop=True)], axis=1)
    df_attention_weights['cross_layer_side'] = df_attention_weights['type'].str.split(': ').str.get(1)
    return df_attention_weights

def get_attention_weights(dataset_test, model, index=0):
    analysis_loader = torch.utils.data.DataLoader(dataset=dataset_test, batch_size=1, shuffle=False)
    for i, (xx,yy) in enumerate(analysis_loader):
        if i==index:
            break
    __xx = [x.to(device) for x in xx]
    #print(__xx)
    explain_model = Explain_TCRModel(d_model=d_model, d_ff=d_ff, n_head=n_head, n_local_encoder=n_local_encoder, 
                                     n_global_encoder=n_global_encoder, dropout=dropout, scope=4, 
                                     n_tok=n_tok, n_pos1=n_pos1, n_pos2=n_pos2, n_seg=n_seg)
    explain_model.load_state_dict(model.state_dict(), strict=False)
    model.to(device)
    explain_model = explain_model.to(device)
    model = model.eval()
    explain_model = explain_model.eval()
    
    with torch.no_grad():
        tcrab, epitope, src_kwargs, tgt_kwargs = explain_model(__xx)  #  (L,B,E)
        attn_output1, attn_output_weights1 = model.enc.cgr.mha.fg.f.mha(
            query=tcrab,  # target  27
            key=epitope,   # source  10  (source is usually the output of encoder)
            value=epitope,   # source  10  
            **{'key_padding_mask': tgt_kwargs['memory_key_padding_mask']}
        )

        attn_output2, attn_output_weights2 = model.enc.cgr.mha.fg.g.f.mha(
            query=epitope, 
            key=tcrab, 
            value=tcrab,
            **{'key_padding_mask': src_kwargs['memory_key_padding_mask']}
        )
        
        ypred = model(__xx)
    return attn_output_weights1, attn_output_weights2, ypred


def vis_attn_to_tcr_from_epitope(attn_output_weights1, tcr_ab_seq, episeq):
    attention_weight = attn_output_weights1[0][:len(tcr_ab_seq), :len(episeq)]
    print((attention_weight.sum(axis=1)))
    x_label = [f'{x}_{i}' for i,x in enumerate(
        list(episeq) + ['<pad>']*(attention_weight.shape[1]-len(episeq)))]
    y_label = [f'{x}_{i}' for i,x in enumerate(
        list(tcr_ab_seq) + ['<pad>']*(attention_weight.shape[0]-len(tcr_ab_seq)))]
    fig = px.imshow(attention_weight, x=x_label, y =y_label,  width=500, height=600)
    fig.update_layout(autosize=False)
    fig.show()

def vis_attn_to_epitope_from_tcr(attn_output_weights2, tcr_ab_seq, episeq):
    attention_weight = attn_output_weights2[0][:len(episeq), :len(tcr_ab_seq)]
    print((attention_weight.sum(axis=1)))
    x_label = [f'{x}_{i}' for i,x in enumerate(
        list(episeq) + ['<pad>']*(attention_weight.shape[0]-len(episeq)))]
    y_label = [f'{x}_{i}' for i,x in enumerate(
        list(tcr_ab_seq) + ['<pad>']*(attention_weight.shape[1]-len(tcr_ab_seq)))]
    fig = px.imshow(attention_weight, x=y_label, y =x_label, width=600, height=600)
    fig.update_layout(autosize=False)
    fig.show()



def remove_UNK_and_take_subset(df_train):
    df_nounk = df_train[df_train['tcra']!='UNK']
    print("df_nounk.value_counts() \n\t ", df_nounk.sign.value_counts().to_dict())
    print("df_nounk.drop_duplicates('tcrb') \n\t ", df_nounk.drop_duplicates('tcrb').sign.value_counts().to_dict())
    return df_nounk

In [4]:
# FNAME ='../../scripts/dir_kicker/logs/files_fake020211027_051219.csv'
# FNAME ='files__all_train_kfold_20211029_031110.csv'

FNAME ='../scripts/dir_kicker/logs/files_20220212_200237_alldata.csv'

INTERGER_SELECTOR = 1
logdf = pd.read_csv(FNAME)
dt = logdf.checkp[INTERGER_SELECTOR].split('/')[-1]

print('dt', dt)

# CHECK_POINT_DIR = f'../../../tcrpred/../checkpoint/{dt}/'
CHECKPOINT_DIR = f's3://sg-playground-kkoyama-temp/tcrpred/checkpoint/{dt}'

OUT_DIR = '../../../tcr_attention_cachedir'

PATH = f"./fakedata_v0.parquet"  # This is for torch.Dataset
KFOLD_I = logdf.knum[INTERGER_SELECTOR].item()
print('KFOLD_I', KFOLD_I)
REPLACE_TUPLE = ('~/jupyter_notebook/user_work/tcrpred/scripts/../../hhyylog/', 
                 '../../../hhyylog/')

checkpoint_files = get_file_paths('sg-playground-kkoyama-temp', 'tcrpred/checkpoint', '', '.pt')
checkpoint_files_withdt = [f for f in checkpoint_files if f'_{dt}_' in f]

CHECKPOINT_PATH = checkpoint_files_withdt[0].split('/')[-1]

# os.system(f'''aws s3 sync {CHECKPOINT_DIR}/ ./../../../checkpoint/{dt}/''')
# os.system(f'''aws s3 sync s3://sg-playground-kkoyama-temp/tcrpred/hhyylog/{dt}_results_attention_tra.pickle ./../../hhyylog/''')


dt 20220211_191954
KFOLD_I 3


In [5]:
checkpoint_files_withdt

['s3://sg-playground-kkoyama-temp/tcrpred/checkpoint/20220211_191954/best_20220211_191954_checkpoint_94_pr_auc_on_one=0.7952.pt',
 's3://sg-playground-kkoyama-temp/tcrpred/checkpoint/20220211_191954/best_20220211_191954_checkpoint_96_pr_auc_on_one=0.7978.pt']

In [6]:
!aws s3 ls s3://sg-playground-kkoyama-temp/tcrpred/checkpoint/{dt}/

2022-03-09 12:48:02   28750105 best_20220211_191951_checkpoint_100_pr_auc_on_one=0.7924.pt
2022-03-09 12:48:03   28750105 best_20220211_191951_checkpoint_99_pr_auc_on_one=0.7901.pt
2022-03-09 12:49:03   28750105 best_20220211_191954_checkpoint_94_pr_auc_on_one=0.7952.pt
2022-03-09 12:49:03   28750105 best_20220211_191954_checkpoint_96_pr_auc_on_one=0.7978.pt


In [7]:
hhfiles = get_file_paths('sg-playground-kkoyama-temp', 'tcrpred/hhyylog', '', '.pickle')
hhfiles_withdt = sorted([f for f in hhfiles if dt in f and f'k{KFOLD_I}' in f])
hhfiles_withdt

['s3://sg-playground-kkoyama-temp/tcrpred/hhyylog/20220211_191954_k3_datasettest.pickle',
 's3://sg-playground-kkoyama-temp/tcrpred/hhyylog/20220211_191954_k3_test_hh.pickle',
 's3://sg-playground-kkoyama-temp/tcrpred/hhyylog/20220211_191954_k3_test_yy.pickle']

# Score of Results

In [9]:
import warnings
import torch, os, sys
warnings.simplefilter(action='ignore')

import numpy as np
import pandas as pd
sys.path.append('../')
sys.path.append('../..')
from attention_extractor import Explain_TCRModel, Explain_Before_Cross
from Bio.PDB import *
from Bio.PDB.PDBParser import PDBParser


display(logdf)



,knum,yyyymmdd,hhpath,yypath,checkp,sh.o.file,sh.o.name
0,1,20220211_191950,~/jupyter_notebook/user_work/tcrpred/scripts/....,~/jupyter_notebook/user_work/tcrpred/scripts/....,~/jupyter_notebook/user_work/tcrpred/scripts/....,~/~/jupyter_notebook/user_work/tcrpred/scripts...,kicker.sh.o205128
1,3,20220211_191954,~/jupyter_notebook/user_work/tcrpred/scripts/....,~/jupyter_notebook/user_work/tcrpred/scripts/....,~/jupyter_notebook/user_work/tcrpred/scripts/....,~/~/jupyter_notebook/user_work/tcrpred/scripts...,kicker.sh.o205130
2,0,20220211_191951,~/jupyter_notebook/user_work/tcrpred/scripts/....,~/jupyter_notebook/user_work/tcrpred/scripts/....,~/jupyter_notebook/user_work/tcrpred/scripts/....,~/~/jupyter_notebook/user_work/tcrpred/scripts...,kicker.sh.o205127
3,2,20220211_191950,~/jupyter_notebook/user_work/tcrpred/scripts/....,~/jupyter_notebook/user_work/tcrpred/scripts/....,~/jupyter_notebook/user_work/tcrpred/scripts/....,~/~/jupyter_notebook/user_work/tcrpred/scripts...,kicker.sh.o205129
4,4,20220211_192004,~/jupyter_notebook/user_work/tcrpred/scripts/....,~/jupyter_notebook/user_work/tcrpred/scripts/....,~/jupyter_notebook/user_work/tcrpred/scripts/....,~/~/jupyter_notebook/user_work/tcrpred/scripts...,kicker.sh.o205131


(175211,) (175211,)
- acc 0.8010170594312
- balanced_acc 0.8610591678937332
- precision on 1:  0.45304500024536665
- precision on 0:  0.9874910149196164
- confusion matrix tn, fp, fn, tp: 112651, 33437, 1427, 27696
- roc_auc_score 0.9289183509705629
- pr_auc_score on 1 0.7290842933535833
- pr_auc_score on 0 0.9847696330980469


None


In [10]:
hh = pdread(hhfiles_withdt[1])
hh_result = hh.apply(softmax, axis=1)
hh_ave = hh_result[1]

yy = pdread(hhfiles_withdt[2])
yy = yy[0]
print(hh_ave.shape, yy.shape)


AA_DICT = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'GLIN': 'Q', 'LYS': 'K',
     'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 'GLY': 'G',
     'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 'ALA': 'A', 'VAL':'V',
     'GLU': 'E', 'TYR': 'Y', 'MET': 'M', 'SEC':'U', 'PYL':'O', 'HOH':'X'}

SEED = 9
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


DEVICE = 'cpu'
from utils import get_infer_report, get_metrics
print(get_infer_report(yy, hh_ave))

In [11]:
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, auc, accuracy_score, average_precision_score, balanced_accuracy_score, precision_score, precision_recall_curve, recall_score
print('''\t\t\t tn, fp, fn, tp    | acc, bacc, precision_1, precision_0, recall, roc_auc ''')
RANGE = 10
for i in range(1,RANGE):
    thh = i/RANGE
    print(f"thh = {thh} \t | ", end='')
    tn, fp, fn, tp, acc, bacc, precision_1, precision_0, recall = get_metrics(yy, hh_ave >= thh)
    

			 tn, fp, fn, tp    | acc, bacc, precision_1, precision_0, recall, roc_auc 
thh = 0.1 	 | 110160, 35928, 1150, 27973 	 | 0.7884, 0.8573, 0.4378, 0.9897, 0.9605, 0.8573
thh = 0.2 	 | 111020, 35068, 1229, 27894 	 | 0.7928, 0.8589, 0.4430, 0.9891, 0.9578, 0.8589
thh = 0.3 	 | 111628, 34460, 1293, 27830 	 | 0.7959, 0.8599, 0.4468, 0.9885, 0.9556, 0.8599
thh = 0.4 	 | 112114, 33974, 1358, 27765 	 | 0.7983, 0.8604, 0.4497, 0.9880, 0.9534, 0.8604
thh = 0.5 	 | 112651, 33437, 1427, 27696 	 | 0.8010, 0.8611, 0.4530, 0.9875, 0.9510, 0.8611
thh = 0.6 	 | 113366, 32722, 1533, 27590 	 | 0.8045, 0.8617, 0.4575, 0.9867, 0.9474, 0.8617
thh = 0.7 	 | 114745, 31343, 1787, 27336 	 | 0.8109, 0.8620, 0.4659, 0.9847, 0.9386, 0.8620
thh = 0.8 	 | 119052, 27036, 2870, 26253 	 | 0.8293, 0.8582, 0.4927, 0.9765, 0.9015, 0.8582
thh = 0.9 	 | 134160, 11928, 9626, 19497 	 | 0.8770, 0.7939, 0.6204, 0.9331, 0.6695, 0.7939


# Attention

In [14]:
CHECKPOINT_PATH

'best_20220211_191954_checkpoint_94_pr_auc_on_one=0.7952.pt'

### Copy attention-mat and checkpoint from s3 to local

In [13]:
torch_check_point_local = '../../checkpoint/'
fro = os.path.join(CHECKPOINT_DIR, CHECKPOINT_PATH)
if not os.path.exists(os.path.join(torch_check_point_local, CHECKPOINT_PATH)):
    print('loading...')
    os.system(f'''aws s3 cp {fro} {torch_check_point_local}''')
    

In [22]:
os.system(f'''aws s3 cp {fro} ./''')

download: s3://sg-playground-kkoyama-temp/tcrpred/checkpoint/20220211_191954/best_20220211_191954_checkpoint_94_pr_auc_on_one=0.7952.pt to ./best_20220211_191954_checkpoint_94_pr_auc_on_one=0.7952.pt


0

In [20]:
!ls -lh {torch_check_point_local}

total 277384
drwxr-xr-x   6 kyoheikoyama  staff   192B Oct 25  2021 20210901_021352
drwxr-xr-x   6 kyoheikoyama  staff   192B Oct 25  2021 20210907_175614
drwxr-xr-x   6 kyoheikoyama  staff   192B Oct 25  2021 20210907_193255
drwxr-xr-x   4 kyoheikoyama  staff   128B Oct 25  2021 20210907_193531
drwxr-xr-x   4 kyoheikoyama  staff   128B Oct 25  2021 20210908_012715
drwxr-xr-x   4 kyoheikoyama  staff   128B Oct 25  2021 20210911_030554
drwxr-xr-x   4 kyoheikoyama  staff   128B Oct 25  2021 20210911_030913
drwxr-xr-x   4 kyoheikoyama  staff   128B Oct 25  2021 20210911_033545
drwxr-xr-x   4 kyoheikoyama  staff   128B Oct 25  2021 20210911_040850
drwxr-xr-x   4 kyoheikoyama  staff   128B Oct 25  2021 20210911_043036
drwxr-xr-x   4 kyoheikoyama  staff   128B Oct 25  2021 20210911_044036
drwxr-xr-x   4 kyoheikoyama  staff   128B Oct 25  2021 20210911_045039
drwxr-xr-x   4 kyoheikoyama  staff   128B Oct 25  2021 20210911_050046
drwxr-xr-x   4 kyoheikoyama  staff   128B Oct 16  2021 20210911_

In [23]:
!gita best_20220211_191954_checkpoint_94_pr_auc_on_one=0.7952.pt

/bin/bash: gita: command not found


In [12]:
# !aws s3 cp s3://sg-playground-kkoyama-temp/tcrpred/hhyylog/{dt}_results_attention_tra.pickle  ../../hhyylog/

In [13]:
import pickle
results_attention_tra = pickle.load(open(f'../../hhyylog/{dt}_results_attention_tra.pickle','rb'))

In [14]:
len(results_attention_tra)

140082

### Restore model

In [18]:
import json, torch
from recipes.model import TCRModel
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.handlers import Checkpoint, DiskSaver, global_step_from_engine
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
device = DEVICE

n_tok = 29  # NUM_VOCAB
n_pos1 = 62  # MAX_LEN_AB (sum of maxlens of a and b)
n_pos2 = 26  # MAX_LEN_Epitope
n_seg = 3

# parameters
with open(f'../hpo_params/optuna_best.json', 'r') as fp:
    hparams = json.load(fp)

d_model, d_ff, n_head,n_local_encoder,n_global_encoder = hparams['d_model'], hparams['d_ff'], hparams['n_head'], hparams['n_local_encoder'], hparams['n_global_encoder']
dropout=hparams['dropout']
early_stopping_target = "pr_auc_on_one"
batch_size = hparams['batch_size']
lr=hparams['lr']

'''Model, optim, trainer'''
model = TCRModel(d_model=d_model, d_ff=d_ff, n_head=n_head, n_local_encoder=n_local_encoder, 
                 n_global_encoder=n_global_encoder, dropout=dropout, scope=4, 
                 n_tok=n_tok, n_pos1=n_pos1,
                n_pos2=n_pos2, n_seg=n_seg,
                )

# Optimizer
optim = torch.optim.Adam(model.parameters(), lr=lr)

# Loss
loss_fn = torch.nn.CrossEntropyLoss(weight=torch.tensor([1.0, 15.0], device=device))

trainer = create_supervised_trainer(
    model=model, optimizer=optim, loss_fn=loss_fn, device=device,
)

## Get the prediction and the best model
to_load = {'model': model, 'optimizer': optim, 'trainer': trainer}

checkpoint = torch.load(os.path.join(torch_check_point_local, CHECKPOINT_PATH), map_location=torch.device(device))

checkpoint['trainer']['seed'] = 9

Checkpoint.load_objects(to_load=to_load, checkpoint=checkpoint)


# Attention Extractor

In [19]:
from recipes.dataset import TCRDataset
import plotly.express as px
from recipes.dataset import Vocab, Dataset

# Main

## Fix Index to get a specific pdb, interaction

In [20]:
def get_values_from_index(ind, df):
    PDBID = df['pdb_id'].iloc[ind]
    DATA_SERIESE = df.iloc[ind]
    e_pdb_chain, tcr_c1_pdb_chain,tcr_c2_pdb_chain,chain1_cdr3_start_calculated,chain1_cdr3_end_calculated,chain2_cdr3_start_calculated, chain2_cdr3_end_calculated =\
                DATA_SERIESE[['e_pdb_chain', 'tcr_c1_pdb_chain','tcr_c2_pdb_chain',
                        'chain1_cdr3_start_calculated', 'chain1_cdr3_end_calculated',
                    'chain2_cdr3_start_calculated', 'chain2_cdr3_end_calculated']]
    return PDBID, e_pdb_chain, tcr_c1_pdb_chain,tcr_c2_pdb_chain,chain1_cdr3_start_calculated,chain1_cdr3_end_calculated,chain2_cdr3_start_calculated, chain2_cdr3_end_calculated


In [21]:
# print(len(DF_TCR.iloc[INDEX]['ABPairSeq']), len(DF_TCR.iloc[INDEX]['EpiSeq']))
# print((DF_TCR.iloc[INDEX]['ABPairSeq']), (DF_TCR.iloc[INDEX]['EpiSeq']))

In [23]:
# explain_model = Explain_TCRModel(d_model=d_model, d_ff=d_ff, n_head=n_head, n_local_encoder=n_local_encoder, 
#                                  n_global_encoder=n_global_encoder, dropout=dropout, scope=4, 
#                                  n_tok=n_tok, n_pos1=n_pos1, n_pos2=n_pos2, n_seg=n_seg)

## Save all

In [15]:
import pickle
import pandas as pd
from sklearn.model_selection import KFold
sys.path.append('./../scripts/')
from dataset_selector import get_df_from_path, get_df


p_list = [
    f"../external_data/ERGO-II/Samples/vdjdb_train_samples.pickle",
    f"../external_data/ERGO-II/Samples/mcpas_train_samples.pickle",
    f"../external_data/ERGO-II/Samples/vdjdb_test_samples.pickle",
    f"../external_data/ERGO-II/Samples/mcpas_test_samples.pickle",]

df_all = get_df_from_path(p_list)
print("df_all.value_counts() \n\t", df_all.sign.value_counts().to_dict())


df_all.value_counts() 
	 {0: 291315, 1: 58263}


In [19]:
df_all['tcra'].unique()[-2:]

array(['CAADRNNPRF', 'CIVAAISGSARQLTF'], dtype=object)

In [16]:
df_all.shape

(349578, 11)

In [35]:
(df_all.tcra!='UNK').sum()

175211

In [39]:
p_list = [
          f"../../external_data/ERGO-II/Samples/vdjdb_train_samples.pickle",
          f"../../external_data/ERGO-II/Samples/mcpas_train_samples.pickle",
          f"../../external_data/ERGO-II/Samples/vdjdb_test_samples.pickle",
          f"../../external_data/ERGO-II/Samples/mcpas_test_samples.pickle",
]

df_all = get_df_from_path(p_list)

# ipdb> df_all.shape
# (349578, 11)
#  len(df_train), len(df_valid)
# (279663, 69915)

print('df_all.shape', df_all.shape)
print("df_all(train).value_counts() \n\t", df_all.sign.value_counts().to_dict())

kf = KFold(n_splits=5, shuffle=True, random_state=2)
train_index, valid_index = [
    (train_index, valid_index) for train_index, valid_index in kf.split(df_all)
][KFOLD_I]

print('len(train_index), len(valid_index)',len(train_index), len(valid_index))
df_train, df_valid = df_all.loc[train_index], df_all.loc[valid_index]

print('len(df_train), len(df_valid)', len(df_train), len(df_valid))
dataset_train, dataset_valid = MCPASDataset(df_train), MCPASDataset(df_valid)


print('len(dataset_train), len(df_train), len(dataset_valid), len(dataset_valid) + len(dataset_train)')
print(len(dataset_train), len(df_train), len(dataset_valid), len(dataset_valid) + len(dataset_train))

df_all.shape (349578, 11)
df_all(train).value_counts() 
	 {0: 291315, 1: 58263}
len(train_index), len(valid_index) 279663 69915
len(df_train), len(df_valid) 279663 69915
datasize and y-vcounts:  (140082, 11) 0    116757
1     23325
Name: sign, dtype: int64
datasize and y-vcounts:  (35129, 11) 0    29331
1     5798
Name: sign, dtype: int64
len(dataset_train), len(df_train), len(dataset_valid), len(dataset_valid) + len(dataset_train)
140082 279663 35129 175211


# DF_TCR and attention_matrix_dict

In [41]:
DF_TCR = df_train

In [45]:
from attention_extractor import Explain_TCRModel, Explain_Before_Cross
attn_output_weights1, attn_output_weights2, ypred = get_attention_weights(dataset_train, model, index=0)

In [47]:
# !aws s3 cp s3://sg-playground-kkoyama-temp/tcrpred/tcr_attention_cachedir/20211029_011638_attention_matrix_dict.pickle ../../tcr_attention_cachedir/

In [50]:
import gc
import pickle
from tqdm import tqdm

NNN = len(dataset_train)

assert NNN==len(df_train)

# if False: #
if os.path.exists(f'{OUT_DIR}/{dt}_attention_matrix_dict.pickle'):
    print('reading attention_matrix_dict.pickle.... \n', f'{OUT_DIR}/{dt}_attention_matrix_dict.pickle')
    ATTENTION_MATRIX_DICT = pickle.load(open(f'{OUT_DIR}/{dt}_attention_matrix_dict.pickle','rb'))
else:
    ATTENTION_MATRIX_DICT = {}
    i=0
    for ind_row, attn in  tqdm(zip(df_train.iterrows(), results_attention_tra)): #tqdm(range(NNN)):
        ind, row = ind_row
        attn_output_weights1, attn_output_weights2, ypred, ytrue = attn #get_attention_weights(dataset_train, model, index=ind)
        ATTENTION_MATRIX_DICT[i] = (attn_output_weights1, 
                                      attn_output_weights2,  
                                      row['tcra'] + ':' + row['tcrb'], 
                                      row['peptide'], 
                                      ypred, 
                                      row['sign'])
        gc.collect()
        i+=1
    pickle.dump(ATTENTION_MATRIX_DICT, open(f'{OUT_DIR}/{dt}_attention_matrix_dict.pickle', 'wb'))
    

140082it [3:13:47, 12.05it/s]
